In [1]:
import cv2
import itertools
import numpy as np

First step is to open the video and store the frames in a list named F

In [2]:
cap = cv2.VideoCapture('tag.mp4')
if (cap.isOpened() == False):
	print("Unable to open the video file")
else:
  print('the video is stored in the cap variable')


the video is stored in the cap variable


STORING THE FRAMES AS PER THE USE

In [3]:
FRAMES = []
while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:
    FRAMES.append(frame)
  else:
    break

print('All frames stored, now showing one of the frames')
cv2.imshow('i', FRAMES[0])
cv2.waitKey()
cv2.destroyAllWindows()

All frames stored, now showing one of the frames


Writing a function to create homography matrix by using the world detect_corners and the image corners as the input

In [4]:
def Get_2DHomography_Matrix(W_frame_corners, Cam_frame_corners):
    world_c = W_frame_corners
    cam_frame_c = Cam_frame_corners
    A_mat = np.zeros((8,9))
    A_mat = np.matrix(A_mat)
    for w,ima in zip(enumerate(world_c), enumerate(cam_frame_c)):
    #here x has the coordinates of world and y has the coordinates of image/camera frame
        i,j,x,y = w[0], ima[0], w[1], ima[1]
        A_mat[i+j,:] = -x[0],-x[1],-1,0,0,0,y[0]*x[0],y[0]*x[1],y[0]
        A_mat[(i+j)+1,:] = 0,0,0,-x[0],-x[1],-1,y[1]*x[0],y[1]*x[1],y[1]
    U, S, V_T = np.linalg.svd(A_mat)
    V_T = np.matrix(V_T)
    V = V_T.T
    H_gr = V[:,-1]
    H_gr = np.reshape(H_gr, [3,3])
    H_final = H_gr/H_gr[-1,-1]
    return H_final
    


Inverse_warping function to project any kind of image using the calculated homography, I have utlized inverse warping to avoid gaps in the image. So, the homography is between the world frame to the camera frame but inverse homography will map it from the camera frame to the world frame. Which agin gave some spots in the image but mean and median blur helped to smoothen out the projection

In [5]:
def Inverse_warping(im, H, size, tes = None):
    #creating all the possible indexes, just ;ike the grid method
    Y_t, X_t = np.indices((size[0], size[1]))
    #flattening of points to create all the possible cooridnates of world frame
    world_frame_pts = np.stack((X_t.ravel(), Y_t.ravel(), np.ones(X_t.size)))

    H_inv = np.linalg.inv(H)
    #conversion of chosen/given world frame to image/camera frame
    image_frame_loc = H_inv.dot(world_frame_pts)
    image_frame_loc /= image_frame_loc[2,:]

    X_image_f, Y_image_f = image_frame_loc[:2,:].astype(int)
    # padding
    X_image_f[X_image_f >=  im.shape[1]] = im.shape[1]
    X_image_f[X_image_f < 0] = 0
    Y_image_f[Y_image_f >=  im.shape[0]] = im.shape[0]
    Y_image_f[Y_image_f < 0] = 0
    
    if (type(tes)==np.ndarray):
        im[Y_image_f, X_image_f, :] = tes[Y_t.ravel(), X_t.ravel(), :]
        return im
    else:
        image_warp = np.zeros((size[0],size[1], 3))
        image_warp[Y_t.ravel(), X_t.ravel(), :]= im[Y_image_f, X_image_f, :]
        return image_warp

An algorithm (referenced from online), which help us in detecting all rectangles in the image with some calibration. It was easy to find rectangles and then going ahead with trying to get the homography and decoding. With more tuning, the best results were taken out using orientatation. The corners are given to this function after detection using Shi-Tomashi method.
The algorithm is referenced in the report which comes from the idea of APRIL lab in U Mich.

In [6]:
def is_rect(detected_corners:list()):
    #an empty matrix to store the pair of points
    pair_o_points = []
    #now creating combination of all points using inbuilt library (8C2)
    for p in itertools.combinations(detected_corners,2):
        a,b= p[0],p[1]
        distance = np.linalg.norm(a-b)
        if(distance>250):
            sl = (b[1]-a[1])/(b[0]-a[0])
            C =  np.array([(b[0]+a[0])/2,(b[1]+a[1])/2])
            pair_o_points.append([a,b,sl,C,distance])
    #now after selecting the specific points we will check whether it's a rectangle using diagonal's centre          
    sorted_corners = np.array(pair_o_points,dtype=object)
    sorted_corners = sorted_corners[np.argsort(sorted_corners[:,-1])][::-1]
    selected_rects = []
    rect_center = []
    #using selected points, now we will make pair of lines and check them
    for lines in itertools.combinations(sorted_corners,2):
        l1,l2 = lines[0],lines[1]
        #checking the difference in length
        if(np.abs(l1[-1]-l2[-1])>5):
            continue
        #checking the intersection of centers
        if (np.linalg.norm(l2[3]-l1[3])>5):
            continue
        rect_center.append(np.array([l1[3],np.linalg.norm(l1[0]-l2[0])]))
        selected_rects.append(np.array([tuple(l1[0].tolist()),tuple(l2[0].tolist()),tuple(l1[1].tolist()),tuple(l2[1].tolist())]))
        
    return selected_rects,rect_center

Next part required was to write an algorithm which can try to decode the tags. It can give undesired orientations and ids, which I filtered out in further steps.

In [7]:
def tag_ID_detector(ima):
    ima = cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    #Consider an image with only two distinct image values (bimodal image), 
    # where the histogram would only consist of two peaks. A good threshold 
    # would be in the middle of those two values. Similarly, Otsu's method 
    # determines an optimal global threshold value from the image histogram.
    
    _,ima = cv2.threshold(ima,127,255,cv2.THRESH_OTSU+cv2.THRESH_OTSU)
    #splitting the frame size of 160 into 8 segments
    GRID = np.array_split(ima,8)
    # print(GRID)
    GRID = np.array(GRID)
    actual_tag = np.zeros((8,8))
    for i,gr in enumerate(GRID):
        for j,intern_grid in  enumerate(np.array_split(gr,8,axis=1)):
            if np.count_nonzero(intern_grid) < (1/2)*intern_grid.size:
                actual_tag[i][j] = 0
            else:
                actual_tag[i][j] =1
    
    if actual_tag[5][5]:
        orientation = 0
    elif actual_tag[2][2]:
        orientation = 180
    elif actual_tag[2][5]:
        orientation = 90
    elif actual_tag[5][2]:
        orientation = 270
    
    else :
        orientation = None
    T_ID = (int(actual_tag[3,3])*1+int(actual_tag[3,4])*2+int(actual_tag[4,4])*4+int(actual_tag[4,3])*8)
    T_ID = (((T_ID<<(orientation//90))&0b1111)|(T_ID>>(4-(orientation//90))))
    if((actual_tag[0:2]!=0).any() and (actual_tag[:][0:2]!=0).any() and (actual_tag[6:-1][:]!=0).any() and (actual_tag[:][6:-1]!=0).any()):
        return np.nan,None,None
    

    return actual_tag,orientation,T_ID

After getting the homography and breaking it into intrinsic and extrinsic parameters and then finding the thrid rotation vector, gives us the possibility to extraction 3D rotational and translation matrix, using which we can find the projection matrix. 

In [8]:
def Transformation_matrix(K, H):
    K_inv = np.linalg.inv(K)
    H1, H2 = H[:,0], H[:,1]
    lam = (np.linalg.norm(K_inv.dot(H1)) + np.linalg.norm(K_inv.dot(H2)) )/2
    lam = 1/lam
    B_tilde = lam*np.dot(K_inv, H)
    if np.linalg.det(B_tilde) > 0 :
        B = B_tilde
    else:
        B = - B_tilde
    R1 = np.array(B[:,0])
    R2 = np.array(B[:,1])
    r3 = np.cross(R1.T, R2.T)
    Trans = B[:,2]
    # print(t.shape)
    Rot = np.hstack([R1,R2,r3.T])

    #P = np.dot(K, M)
    return Rot, Trans

By multiplying each of the decided vertex for the cube, the cube can be easily projected onto the AR tag

In [9]:
def project_cube(proj_corner, img): 
    #defining the thickness of cube
    thickness = 5
    #making a copy of an image 
    proj_cube_im = np.copy(img)
    #segmenting the cube vetex into higher and lower plain
    l_p, h_p = np.array_split(proj_corner,2)
    h_p = np.resize(h_p,(4,2))
    l_p = np.resize(l_p,(4,2))
    #drawing the upper and lower line of planes
    proj_cube_im = cv2.polylines(proj_cube_im,[l_p],True,(255,0,0),thickness)
    proj_cube_im = cv2.polylines(proj_cube_im,[h_p],True,(255,0,0),thickness)
    #for joining the vertex of the cubes
    for n in zip(l_p,h_p):
        lower,upper = n
        proj_cube_im = cv2.polylines(proj_cube_im,[np.array([lower,upper])],True,(255,0,0),thickness)
    return proj_cube_im

In [10]:
def corner_detection(ima):
    ima = cv2.medianBlur(ima, 7) # Blur the image slightly
    gray = cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY) #convert to grayscale

    #Perform corner detection
    det_corners = cv2.goodFeaturesToTrack(gray,25,0.35,10) 
    det_corners = np.int0(det_corners)
    list_of_corners = []
    for i in det_corners:
        x,y = i.ravel()
        list_of_corners.append(np.array([x,y]))
        cv2.circle(img, (x,y), 3, 255, -1)
    return list_of_corners

The main program starts from here

In [16]:
Num = 0
img_c = np.copy(FRAMES[Num]) #assign the frame of interest
img = np.copy(FRAMES[Num])
corner = []
corner = corner_detection(img)
## detect all the rectangle in the image
rectangles,d=is_rect(corner)
AR_tag = []
world_homo_frame = 160 #can be equal to detected edge
for i, rect_corn in enumerate(rectangles):
    world_frame_corn = [[0,0],[0,world_homo_frame],[world_homo_frame,world_homo_frame],[world_homo_frame,0]]
    homo = Get_2DHomography_Matrix(rect_corn,world_frame_corn)
    # world_frame_img = cv.warpPerspective(frames[0], homo,(world_homo_frame,world_homo_frame))
    world_frame_img = np.uint8(Inverse_warping(img, homo,(world_homo_frame,world_homo_frame)))
    homo2 = Get_2DHomography_Matrix(world_frame_corn,world_frame_corn)
    #to compensate for the mirror image
    world_frame_img = np.uint8(Inverse_warping(np.copy(world_frame_img), homo2,(world_homo_frame,world_homo_frame)))

    cv2.imshow("world_frame_img",world_frame_img)
    cv2.waitKey()
    cv2.destroyAllWindows()
    try:
        actual_tag,orientation,T_ID = tag_ID_detector(world_frame_img)
    except:
        continue
    AR_tag.append([actual_tag,orientation,T_ID])
    print('tag id is', T_ID)

/home/rishabh/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


tag id is 7


In [17]:
    T_ID

7

In [18]:
    orientation

90

In [19]:
    testudo = cv2.imread('test.png')
    testudo = cv2.resize(testudo, (world_homo_frame, world_homo_frame))
    testudo = np.rot90(testudo)

In [20]:
    a = Inverse_warping(img, homo, (int(world_homo_frame),int(world_homo_frame)), testudo)
    cv2.imshow('im', np.uint8(a))
    cv2.waitKey()
    cv2.destroyAllWindows()



Instead of interpolation, the median blur worked really well to smoothen out the image

In [21]:
    a = cv2.medianBlur(a, 3)
    cv2.imshow('im', np.uint8(a))
    cv2.waitKey()
    cv2.destroyAllWindows()

Intrinsic matrix

In [22]:
    In_param = np.matrix([[1346.100595, 0, 932.1633975], [0, 1355.933136, 654.8986796], [0, 0, 1]])

Next part was to get the projection matrix and the vertex of the cube in the image frame 

In [23]:
    p1,p2 = Transformation_matrix(In_param, np.linalg.inv(homo))


In [24]:
    points_to_project = np.float32([[0, 0, 0], 
                    [160, 0, 0], 
                    [160, 160, 0], 
                    [0, 160, 0], 
                    [0, 0, -160], 
                    [160, 0, -160], 
                    [160, 160, -160],
                    [0, 160, -160]])



In [25]:
    projected_corners, _ = cv2.projectPoints(points_to_project, p1, p2, In_param, np.zeros((1, 4)))
    pts = np.int32(projected_corners)
    f = np.copy(img_c)

In [26]:
    c = project_cube(pts, f)
    cv2.imshow('image', np.uint8(c))
    cv2.waitKey(0)
    cv2.destroyAllWindows()